In [1]:
import pandas as pd
import os

def find_bug(df):
    """
    df_offers имеет странные дублирующиеся model_id_pred. Векроятно это баг в функции top_n_similar 
    (не находит достаточное количество до n шт матчей + дальнейший convert_columns_to_int)
    """
    columns_to_check = ['model_id_pred_1', 'model_id_pred_2', 'model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
    duplicate_rows_indices = df[df[columns_to_check].apply(lambda row: row.duplicated().sum() >= 1, axis=1)].index
    new_df = df.loc[duplicate_rows_indices]
    return new_df

def merge_columns_to_list(df, n, column_name):
    """
    Функция для объединения значений из нескольких колонок в один список.
    
    Параметры:
        df (DataFrame): Исходный DataFrame.
        n (int): Количество колонок, которые нужно объединить.
        column_name (str): Имя обрабатываемых колонок.
        
    Возвращает:
        DataFrame: DataFrame с добавленным столбцом, содержащим объединенные значения в виде списка.
    """
    # Создаем список колонок для объединения
    columns = [f"{column_name}_{i}" for i in range(1, n+1)]
    
    # Объединяем значения в список
    df[column_name] = df[columns].apply(lambda row: row.tolist(), axis=1)
    
    return df

pth_models = "/home/sondors/Documents/price/ColBERT_data/18_categories/test/models_18_categories.csv"
pth_offers = "/home/sondors/Documents/price/ColBERT/proverka/colbert-5387_offers_top_n.csv"

df_models = pd.read_csv(pth_models, sep=";")
# df_models = df_models.drop(columns=['average_price', 'comment'])
df_offers = pd.read_csv(pth_offers, sep=";")
# df_offers = df_offers.drop(columns=['true_match', 'false_match'])


df_offers = merge_columns_to_list(df_offers, 5, "model_id_pred")
df_offers = merge_columns_to_list(df_offers, 5, "similarity")
df_offers

,name,model_id,category_name,category_id,model_id_pred_1,similarity_1,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_correct,model_id_pred,similarity
0,Портативная акустика Hoco HC2 Xpress темно-зел...,4771613,портативная акустика,3904,4771613,29.91,4851243,27.94,4777265,22.92,4851242,22.30,5491278,21.59,NaN,"[4771613, 4851243, 4777265, 4851242, 5491278]","[29.91, 27.94, 22.92, 22.3, 21.59]"
1,"Смартфон Xiaomi POCO M5 4/128 GB Global, серый",5506115,мобильные телефоны,2801,5506115,27.33,5667654,26.23,5667653,21.38,5254579,21.25,5506116,21.23,NaN,"[5506115, 5667654, 5667653, 5254579, 5506116]","[27.33, 26.23, 21.38, 21.25, 21.23]"
2,"Портативная колонка Ginzzu GM-906B, BT-Колонка...",4585102,портативная акустика,3904,4585102,28.42,1017511,24.23,939517,24.12,1431431,23.80,3033277,23.39,NaN,"[4585102, 1017511, 939517, 1431431, 3033277]","[28.42, 24.23, 24.12, 23.8, 23.39]"
3,Часы Amazfit BIP S Black,4565835,смарт-часы и браслеты,2815,4565835,28.23,4565836,23.02,4565837,19.52,3390599,19.11,5418954,18.05,NaN,"[4565835, 4565836, 4565837, 3390599, 5418954]","[28.23, 23.02, 19.52, 19.11, 18.05]"
4,"JBL Портативная акустика JBL Charge 5, розовый",4771627,портативная акустика,3904,4771627,29.80,327183,25.20,1018852,24.19,4048259,23.91,682908,23.81,NaN,"[4771627, 327183, 1018852, 4048259, 682908]","[29.8, 25.2, 24.19, 23.91, 23.81]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74134,Портативная беспроводная колонка / акустическа...,5128645,портативная акустика,3904,4851954,17.31,5491286,17.14,4851953,17.03,6957048,17.00,6957050,16.89,1.0,"[4851954, 5491286, 4851953, 6957048, 6957050]","[17.31, 17.14, 17.03, 17.0, 16.89]"
74135,Универсальная мобильная гарнитура JBL C50HI RE...,4455819,"наушники, гарнитуры, наушники c микрофоном",2102,746901,15.19,4300186,15.00,616609,14.67,760225,14.57,4300180,14.42,1.0,"[746901, 4300186, 616609, 760225, 4300180]","[15.19, 15.0, 14.67, 14.57, 14.42]"
74136,Планшетный компьютер Digma Планшет Citi 7591 3...,4273003,планшетные компьютеры и мини-планшеты,510401,3843911,25.70,4273002,25.31,1950145,25.25,4404425,25.22,2341722,25.12,1.0,"[3843911, 4273002, 1950145, 4404425, 2341722]","[25.7, 25.31, 25.25, 25.22, 25.12]"
74137,Портативная акустическая система Hyundai Колон...,4570004,портативная акустика,3904,4569993,24.36,4569996,24.23,4569999,24.23,3709109,24.17,4585106,24.17,1.0,"[4569993, 4569996, 4569999, 3709109, 4585106]","[24.36, 24.23, 24.23, 24.17, 24.17]"


In [2]:
def accuracy_top_n(offer_model_id, offer_model_id_pred, top_k):
    correct_predictions = 0

    for true_id, predicted_ids in zip(offer_model_id, offer_model_id_pred):
        top_k_predictions = predicted_ids[:top_k]
        if true_id in top_k_predictions:
            correct_predictions += 1

    accuracy = correct_predictions / len(offer_model_id)
    return accuracy

def calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k): # проверить функцию
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for true_id, predicted_ids in zip(offer_model_id, offer_model_id_pred):
        top_k_predictions = predicted_ids[:top_k]
        
        if true_id in top_k_predictions:
            true_positives += 1
        else:
            false_negatives += 1
            false_positives += (true_id in predicted_ids)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return precision, recall

def calc_acc_for_category(df_offers, df_models, imya_kategorii):
    print(f"{imya_kategorii}")
    df_offers = df_offers[df_offers.category_name == imya_kategorii]
    df_models = df_models[df_models.category_name == imya_kategorii]

    offer_names = list(df_offers['name'])
    offer_model_id = list(df_offers['model_id'])
    offer_model_id_pred = list(df_offers['model_id_pred'])

    model_full_names = list(df_models['full_name'])

    print(f"\tофферов {len(offer_model_id)} шт поиск матча для каждого среди {len(model_full_names)} шт моделей:\n")

    top1_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 1)
    top2_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 2)
    top3_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 3)
    top4_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 4)
    top5_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 5)

    top1_precision, top1_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=1)
    top3_precision, top3_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=3)
    top5_precision, top5_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=5)

    print(f'\tTop-1: Accuracy = {top1_accuracy:.2%}')
    print(f'\tTop-3: Accuracy = {top3_accuracy:.2%}')
    print(f'\tTop-5: Accuracy = {top5_accuracy:.2%}\n')

    # print(f'\tTop-1: Accuracy = {top1_accuracy:.2%}, Precision = {top1_precision:.2%}, Recall = {top1_recall:.2%}')
    # print(f'\tTop-3: Accuracy = {top3_accuracy:.2%}, Precision = {top3_precision:.2%}, Recall = {top3_recall:.2%}')
    # print(f'\tTop-5: Accuracy = {top5_accuracy:.2%}, Precision = {top5_precision:.2%}, Recall = {top5_recall:.2%}\n')
    return (top1_accuracy, top2_accuracy, top3_accuracy, top4_accuracy, top5_accuracy)

id_category = {
    3902: 'диктофоны, портативные рекордеры',
    510402: 'электронные книги',
    4302: 'автомобильные телевизоры, мониторы',
    2815: 'смарт-часы и браслеты',
    3901: 'портативные медиаплееры',
    3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    510401: 'планшетные компьютеры и мини-планшеты',
    2102: 'наушники, гарнитуры, наушники c микрофоном',
    3903: 'радиоприемники, радиобудильники, радиочасы',
    3907: 'магнитолы',
    280801: 'GPS-навигаторы'
    }

for category in id_category.values():
    accuracies_top_n = calc_acc_for_category(df_offers, df_models, category)

диктофоны, портативные рекордеры
	офферов 248 шт поиск матча для каждого среди 488 шт моделей:

	Top-1: Accuracy = 95.16%
	Top-3: Accuracy = 99.19%
	Top-5: Accuracy = 100.00%

электронные книги
	офферов 361 шт поиск матча для каждого среди 680 шт моделей:

	Top-1: Accuracy = 90.86%
	Top-3: Accuracy = 98.89%
	Top-5: Accuracy = 99.45%

автомобильные телевизоры, мониторы
	офферов 84 шт поиск матча для каждого среди 790 шт моделей:

	Top-1: Accuracy = 85.71%
	Top-3: Accuracy = 92.86%
	Top-5: Accuracy = 95.24%

смарт-часы и браслеты
	офферов 5969 шт поиск матча для каждого среди 2577 шт моделей:

	Top-1: Accuracy = 93.70%
	Top-3: Accuracy = 98.58%
	Top-5: Accuracy = 99.73%

портативные медиаплееры
	офферов 580 шт поиск матча для каждого среди 847 шт моделей:

	Top-1: Accuracy = 92.59%
	Top-3: Accuracy = 98.62%
	Top-5: Accuracy = 99.48%

портативная акустика
	офферов 12614 шт поиск матча для каждого среди 3800 шт моделей:

	Top-1: Accuracy = 93.52%
	Top-3: Accuracy = 97.25%
	Top-5: Accuracy 

In [12]:
def write_errors(df_offers, df, df_models, dst_fld, id_category):
    os.makedirs(dst_fld, exist_ok=True)
    for category in id_category.values():
        category_error = df[df.category_name == category]
        category_all = df_offers[df_offers.category_name == category]
        accuracies_top_n = calc_acc_for_category(category_error, df_models, category)
        with open(os.path.join(dst_fld, f"{category}.txt"), "a") as txt:
            txt.write(f'Всего офферов {len(category_all)} шт из них ошибок {len(category_error)} шт\nРаспределение ошибок:\n')
            txt.write(f'\nTop-1: Accuracy = {accuracies_top_n[0]:.2%}')
            txt.write(f'\nTop-2: Accuracy = {accuracies_top_n[1]:.2%}')
            txt.write(f'\nTop-3: Accuracy = {accuracies_top_n[2]:.2%}')
            txt.write(f'\nTop-4: Accuracy = {accuracies_top_n[3]:.2%}')
            txt.write(f'\nTop-5: Accuracy = {accuracies_top_n[4]:.2%}\n\n')
            txt.write("Истинная модель:\nПоле name оффера --> Найденная модель топ1")
            txt.write("\n____________________________________________________________\n")
            for index, row in category_error.iterrows():
                model_gt = list(df_models[df_models.model_id == row['model_id']]['full_name'])[0]
                model_pred = list(df_models[df_models.model_id == row['model_id_pred_1']]['full_name'])[0]
                txt.write(f"\n{model_gt}:\n{row['name']} --> {model_pred}\n")                
def exclude_from_df(df, col):
    return df[df['model_id'] != df[col]].reset_index(drop=True)

clms = ['model_id_pred_1','model_id_pred_2','model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
df = df_offers.copy()
for i in range(len(clms)):
    df = exclude_from_df(df, clms[i])
    dst_fld = f"/home/sondors/Documents/price/ColBERT/EVAL/errors_ColBERT/exclude_top_{i+1}"
    write_errors(df_offers, df, df_models, dst_fld, id_category)

диктофоны, портативные рекордеры
	офферов 15 шт поиск матча для каждого среди 488 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 66.67%
	Top-5: Accuracy = 80.00%

электронные книги
	офферов 39 шт поиск матча для каждого среди 680 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 74.36%
	Top-5: Accuracy = 79.49%

автомобильные телевизоры, мониторы
	офферов 28 шт поиск матча для каждого среди 790 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 21.43%
	Top-5: Accuracy = 28.57%

смарт-часы и браслеты
	офферов 490 шт поиск матча для каждого среди 2577 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 59.39%
	Top-5: Accuracy = 73.47%

портативные медиаплееры
	офферов 52 шт поиск матча для каждого среди 847 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 67.31%
	Top-5: Accuracy = 76.92%

портативная акустика
	офферов 1480 шт поиск матча для каждого среди 3800 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 31.82%
	Top-5: Accuracy = 40.07%

мо